In [52]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import WazeRouteCalculator
import datetime

## Pulling Job Data

In [53]:
project_path = "/Users/jjespinoza/GoogleDrive/2_projects/2_itsfriday/"
#project_path = "/home/jj_espinoza_la/itsfriday/"

now = datetime.datetime.now()
now.year
now.month
now.day
date = str(now.year) + str(now.month) + str(now.day) + "_" + str(now.hour) + str(now.minute)
date


'2018121_1140'

In [104]:
max_results_per_city = 100
max_commute = 45
#city_set = ['Los+Angeles']
city_set = ['Los+Angeles', 'Los+Angeles+County', 'Long+Beach', 'Downey', 'Commerce']
#title_set = ['data+scientist']
title_set = ['data+scientist', 'senior+data+scientist', 'director+data+science', 'director+analytics', 'vice+president+analytics', 'vice+president+data+science']
columns = ['city', 'job_title', 'company_name', 'location', 'summary', 'salary']
sample_df = pd.DataFrame(columns = columns)


In [55]:
#scraping code:
for title in title_set:
    for city in city_set:
        for start in range(0, max_results_per_city, 10):
            page = requests.get('http://www.indeed.com/jobs?q='+ str(title) +'+%2420%2C000&l=' + str(city) + '&start=' + str(start))
            time.sleep(1)  #ensuring at least 1 second between page grabs
            soup = BeautifulSoup(page.text, 'lxml')
            for div in soup.find_all(name='div', attrs={'class':'row'}): 
                #specifying row num for index of job posting in dataframe
                num = (len(sample_df) + 1) 
                #creating an empty list to hold the data for each posting
                job_post = [] 
                #append city name
                job_post.append(city) 
                #grabbing job title
                for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
                    job_post.append(a['title']) 
                #grabbing company name
                company = div.find_all(name='span', attrs={'class':'company'}) 
                if len(company) > 0: 
                    for b in company:
                        job_post.append(b.text.strip()) 
                else: 
                    sec_try = div.find_all(name='span', attrs={'class':'result-link-source'})
                    for span in sec_try:
                        job_post.append(span.text) 
                #grabbing location name
                c = div.findAll('span', attrs={'class': 'location'}) 
                for span in c: 
                    job_post.append(span.text) 
                #grabbing summary text
                d = div.findAll('span', attrs={'class': 'summary'}) 
                for span in d:
                    job_post.append(span.text.strip()) 
                #grabbing salary
                try:
                    job_post.append(div.find('nobr').text) 
                except:
                    try:
                        div_two = div.find(name='div', attrs={'class':'sjcl'}) 
                        div_three = div_two.find('div') 
                        job_post.append(div_three.text.strip())
                    except:
                        job_post.append('Nothing_found') 
                #appending list of job post info to dataframe at index num
                sample_df.loc[num] = job_post




In [63]:
del sample_df['city']

KeyError: 'city'

In [79]:
sample_df['job_title'] = sample_df['job_title'].str.lower()
sample_df['company_name'] = sample_df['company_name'].str.lower()
sample_df['location'] = sample_df['location'].str.lower()
sample_df['summary'] = sample_df['summary'].str.lower()
sample_df = sample_df.drop_duplicates(subset=['job_title', 'company_name'], keep = False)

### Saving Raw Data

In [80]:
sample_df.to_csv(project_path + "2_data/1_raw/scraper_indeed_jobs_raw.csv", encoding='utf-8')
sample_df

,job_title,company_name,location,summary,salary
7,data scientist,stackyork,"los angeles, ca",teach and mentor other analysts about good dat...,Nothing_found
11,"associate, data scientist/statistics - financi...",kpmg,"los angeles, ca 90071","machine learning, data visualization, statisti...",Nothing_found
12,data scientist - tech start up,next trucking,"los angeles, ca",we are looking for a motivated data scientist ...,Nothing_found
15,principal data scientist,sartorius,california,sartorius - principal engineer/principal data ...,Nothing_found
37,data scientist – machine learning,internet brands,"los angeles, ca",is currently seeking a data scientist - machin...,Nothing_found
39,financial data scientist,verity business services,"los angeles, ca",the primary function of the data scientist is ...,Nothing_found
50,senior data scientist,veear projects,"los angeles, ca",a history of working with medical claims data....,Nothing_found
54,data entry & analysis clerk (entry level),"motion picture compliance solutions, inc.","sylmar, ca 91342",strong computer skills in the areas data entry...,Nothing_found
57,data science - marketplace optimization & fore...,honk,"los angeles, ca 90064",a proven data scientist or engineer with hands...,Nothing_found
78,senior data scientist,honey,"los angeles, ca",helping mentor more junior data scientists on ...,Nothing_found


## Dropping unwanted job titles


In [87]:
#Try this
sample_df = sample_df[(sample_df["job_title"] != "account") & 
                       (~sample_df["job_title"].str.contains('engineer')) &
                      (~sample_df["job_title"].str.contains('research')) &
                     (~sample_df["job_title"].str.contains('manager')) &
                      (~sample_df["job_title"].str.contains('developer')) & 
                        (~sample_df["job_title"].str.contains('vice president of sales')) & 
                        (~sample_df["job_title"].str.contains('communications')) & 
                    (~sample_df["job_title"].str.contains('client')) & 
                     (~sample_df["job_title"].str.contains('analyst'))]






sample_df

,job_title,company_name,location,summary,salary
7,data scientist,stackyork,"los angeles, ca",teach and mentor other analysts about good dat...,Nothing_found
11,"associate, data scientist/statistics - financi...",kpmg,"los angeles, ca 90071","machine learning, data visualization, statisti...",Nothing_found
12,data scientist - tech start up,next trucking,"los angeles, ca",we are looking for a motivated data scientist ...,Nothing_found
15,principal data scientist,sartorius,california,sartorius - principal engineer/principal data ...,Nothing_found
37,data scientist – machine learning,internet brands,"los angeles, ca",is currently seeking a data scientist - machin...,Nothing_found
39,financial data scientist,verity business services,"los angeles, ca",the primary function of the data scientist is ...,Nothing_found
50,senior data scientist,veear projects,"los angeles, ca",a history of working with medical claims data....,Nothing_found
54,data entry & analysis clerk (entry level),"motion picture compliance solutions, inc.","sylmar, ca 91342",strong computer skills in the areas data entry...,Nothing_found
57,data science - marketplace optimization & fore...,honk,"los angeles, ca 90064",a proven data scientist or engineer with hands...,Nothing_found
78,senior data scientist,honey,"los angeles, ca",helping mentor more junior data scientists on ...,Nothing_found


## Dropping Unwanted Cities

In [93]:
sample_df = sample_df[ (~sample_df["location"].str.contains('woodland hills')) &
                      (~sample_df["location"].str.contains('santa monica')) &
                      (~sample_df["location"].str.contains('culver city')) &
                      (~sample_df["location"].str.contains('burbank')) &
                      (~sample_df["location"].str.contains('hollywood')) &
                      (~sample_df["location"].str.contains('westwood')) &
                      (~sample_df["location"].str.contains('universal city')) &
                      (~sample_df["location"].str.contains('beverly hills')) &
                      (~sample_df["location"].str.contains('playa vista')) &
                      (~sample_df["location"].str.contains('valencia')) &
                      (~sample_df["location"].str.contains('encino')) &
                      (~sample_df["location"].str.contains('venice'))
                     ]



sample_df

,job_title,company_name,location,summary,salary
7,data scientist,stackyork,"los angeles, ca",teach and mentor other analysts about good dat...,Nothing_found
11,"associate, data scientist/statistics - financi...",kpmg,"los angeles, ca 90071","machine learning, data visualization, statisti...",Nothing_found
12,data scientist - tech start up,next trucking,"los angeles, ca",we are looking for a motivated data scientist ...,Nothing_found
15,principal data scientist,sartorius,california,sartorius - principal engineer/principal data ...,Nothing_found
37,data scientist – machine learning,internet brands,"los angeles, ca",is currently seeking a data scientist - machin...,Nothing_found
39,financial data scientist,verity business services,"los angeles, ca",the primary function of the data scientist is ...,Nothing_found
50,senior data scientist,veear projects,"los angeles, ca",a history of working with medical claims data....,Nothing_found
54,data entry & analysis clerk (entry level),"motion picture compliance solutions, inc.","sylmar, ca 91342",strong computer skills in the areas data entry...,Nothing_found
57,data science - marketplace optimization & fore...,honk,"los angeles, ca 90064",a proven data scientist or engineer with hands...,Nothing_found
78,senior data scientist,honey,"los angeles, ca",helping mentor more junior data scientists on ...,Nothing_found


## Dropping Companies

In [97]:
sample_df = sample_df[ (~sample_df["company_name"].str.contains('ucla Extension')) &
                      (~sample_df["company_name"].str.contains('usc')) &
                      (~sample_df["company_name"].str.contains('health')) &
                      (~sample_df["company_name"].str.contains('riot games')) &
                      (~sample_df["company_name"].str.contains('los angeles county')) &
                      (~sample_df["company_name"].str.contains('lieberman research worldwide')) &
                      (~sample_df["company_name"].str.contains('hospital')) &
                      (~sample_df["company_name"].str.contains('medicine')) &
                       (~sample_df["company_name"].str.contains('university')) &
                      (~sample_df["company_name"].str.contains('first 5 la'))
                      
                     ]



health


sample_df

,job_title,company_name,location,summary,salary
7,data scientist,stackyork,"los angeles, ca",teach and mentor other analysts about good dat...,Nothing_found
11,"associate, data scientist/statistics - financi...",kpmg,"los angeles, ca 90071","machine learning, data visualization, statisti...",Nothing_found
12,data scientist - tech start up,next trucking,"los angeles, ca",we are looking for a motivated data scientist ...,Nothing_found
15,principal data scientist,sartorius,california,sartorius - principal engineer/principal data ...,Nothing_found
37,data scientist – machine learning,internet brands,"los angeles, ca",is currently seeking a data scientist - machin...,Nothing_found
39,financial data scientist,verity business services,"los angeles, ca",the primary function of the data scientist is ...,Nothing_found
50,senior data scientist,veear projects,"los angeles, ca",a history of working with medical claims data....,Nothing_found
54,data entry & analysis clerk (entry level),"motion picture compliance solutions, inc.","sylmar, ca 91342",strong computer skills in the areas data entry...,Nothing_found
57,data science - marketplace optimization & fore...,honk,"los angeles, ca 90064",a proven data scientist or engineer with hands...,Nothing_found
78,senior data scientist,honey,"los angeles, ca",helping mentor more junior data scientists on ...,Nothing_found


In [98]:
sample_df.to_csv(project_path + "2_data/2_clean/scraper_indeed_jobs_clean.csv", encoding='utf-8')

### Saving Clean Data

## Calculating Distance

In [99]:
sample_df["clean_address"] = sample_df["company_name"] + ' ' + sample_df["location"]

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
commutes = []

for address in sample_df["clean_address"]:
    from_address = "10757 Longworth Ave Santa Fe Springs CA 90670"
    region = 'US'
    try:
        to_address = address
        route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region )
        commutes.append(route.calc_route_info(real_time=False))
    except:
        commutes.append((0,0))
    
    
        
    
        

From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: stackyork los angeles, ca
Time 47.67 minutes, distance 45.67 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: kpmg los angeles, ca 90071
Time 47.67 minutes, distance 45.67 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: next trucking los angeles, ca
Time 47.67 minutes, distance 45.67 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: sartorius california
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: internet brands los angeles, ca
Time 45.35 minutes, distance 65.83 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: verity business services los angeles, ca
Time 19.42 minutes, distance 20.80 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: veear projects los angeles, ca
Time 19.03 minutes, distance 19.53 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: motion picture compliance solutions, inc. sylmar, ca 91342
From: 10757 Longworth Ave Santa Fe 

Time 45.97 minutes, distance 60.22 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: public health institute monrovia, ca 91016
Time 328.33 minutes, distance 621.89 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: pfizer los angeles, ca
Time 46.88 minutes, distance 45.53 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: universities space research association palmdale, ca
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: simulations plus, inc. lancaster, ca 93534
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: simulations plus, inc. lancaster, ca 93534
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: antelope valley hospital lancaster, ca
Time 84.53 minutes, distance 133.16 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: antelope valley hospital lancaster, ca
Time 84.53 minutes, distance 133.16 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: presbyterian healthcare services united states
Time 63

Time 8.55 minutes, distance 4.66 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: analytic recruiting los angeles, ca
Time 47.85 minutes, distance 45.67 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: american advisors group orange, ca
Time 21.63 minutes, distance 28.93 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: campbell-ewald los angeles, ca 90064
Time 33.53 minutes, distance 40.32 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: kiln los angeles, ca
Time 46.17 minutes, distance 57.90 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: the giving keys los angeles, ca 90013
Time 20.08 minutes, distance 21.61 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: possible los angeles, ca
Time 47.85 minutes, distance 45.67 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: pih health whittier, ca
Time 22.35 minutes, distance 15.45 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: agilon hea

From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: stroz friedberg, an aon company los angeles, ca
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: kaufman hall el segundo, ca 90245
Time 27.93 minutes, distance 38.10 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: beauty brand los angeles, ca
Time 45.02 minutes, distance 65.83 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: mufg monterey park, ca
Time 20.68 minutes, distance 25.82 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: toms shoes los angeles, ca 90066
Time 48.77 minutes, distance 46.07 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: jukin media los angeles, ca
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: sada systems inc. los angeles, ca
Time 40.25 minutes, distance 48.98 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 - to: spokeo pasadena, ca
Time 31.37 minutes, distance 44.63 km.
From: 10757 Longworth Ave Santa Fe Springs CA 90670 -

In [108]:
df = pd.DataFrame(commutes) 
df.columns = ['commute', 'distance_km']
df.index = range(1,len(df)+1)
sample_df = sample_df.join(df, lsuffix='_caller', rsuffix='_other')
sample_df


,city,job_title,company_name,location,summary,salary,commute,distance_km


### Filtering Based on Distance

In [109]:
sample_df.to_csv(project_path + "2_data/2_clean/scraper_indeed_jobs_clean_enhanced.csv", encoding='utf-8')